In [ ]:
import os
import yaml

import cv2
import foolbox as fb
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import tensorflow as tf

from adversarial_generators.fgsm import generate_adversarial_images
from model.VGG19 import VGG19
from preprocess.preprocess import load_data

# Config

In [ ]:
with open('config/config.yml', 'r') as stream:
    config = yaml.safe_load(stream)

ROOT_DIRECTORY = os.path.dirname(os.path.abspath('__file__'))

# Load weights to model

In [ ]:
input_shape = (config["img_height"], config["img_width"], 3)
model = VGG19(input_shape = input_shape, num_classes = config["num_classes"])
model.load_weights(config["path_to_weights"])

# Compile Model

In [ ]:
optimizer = tf.keras.optimizers.Adam(lr = config["learning_rate"], decay = config["learning_rate"] / (config["epochs"]))
loss = config["loss_function"]
metrics = config["metrics"]
model.compile(optimizer = optimizer, loss = loss, metrics = [metrics])

# Load data

In [ ]:
path_to_set = os.path.join(ROOT_DIRECTORY, config["path_to_data"])
path_to_train_csv = os.path.join(ROOT_DIRECTORY, config["path_to_train_csv"])
path_to_test_csv = os.path.join(ROOT_DIRECTORY, config["path_to_test_csv"])
(X_train, y_train) = load_data(path_to_train_csv, path_to_set, config["img_width"], config["img_height"])

# Take random 100 images

In [ ]:
idxs = np.random.randint(low = 0, high = len(X_train), size = 100)
X_train, y_train = X_train[idxs], y_train[idxs]

# Normalize the data

In [ ]:
X_train = X_train.astype("float32") / 255.0

# *Adversarial Training*

In [ ]:
X_adversarial_train = tf.convert_to_tensor(X_train)
y_adversarial_train = tf.convert_to_tensor(y_train)
from foolbox.distances import LpDistance
import eagerpy as ep

# attack = fb.attacks.InversionAttack(distance = LpDistance(2))
attack = fb.attacks.L2ClippingAwareAdditiveGaussianNoiseAttack()
bounds = (0, 1)
preprocessing = dict()
fmodel = fb.TensorFlowModel(model, bounds=bounds, preprocessing=preprocessing)

## Generate Adversarial Images and Plot

In [ ]:
eps = 0.031
_, adversarial_images, indexes_of_wrong_images = attack(fmodel, X_adversarial_train, y_adversarial_train, epsilons = eps)

# Before

In [ ]:
fb.plot.images(X_train, n = 5, scale = 2)

# After

In [ ]:
fb.plot.images(adversarial_images, n = 5, scale = 2)